# Przetwarzanie danych w językach R i Python - praca domowa 4

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<center><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form><c/enter>''')

*Szymon Adach *           
*Jacek Dziwulski*

24 maj 2017

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
py.sign_in('orgiele', '0hLF1CVCaH5sEU0CG9Ah')

# Wstęp

Zadanie polegało na zebraniu danych, dostępnych poprzed odpowiednie endpointy, których szczegóły można znaleźć na stronie https://api.um.warszawa.pl/. Dane te dotyczą ruchu tramwajów oraz autobusów w Warszawie. Do zbierania tych danych, utworzony został specjalny skrypt w języku Python, który co wyznaczony czas pobierał dane z API, oraz zapisywał je do pliku. Jego implementacja znajduje się poniżej.

```python
import time
import helpers
from Lines import VehicleType
import datetime
import os

TIMEOUT = 10.0  # Ten seconds

MEGABYTE = 1024 * 1024
MAX_FILE_SIZE = 2 * 4096 * MEGABYTE #8GB

bus_file_name = "buses3.json"
tram_file_name = "trams3.json"

def saveJsonToFile(tramfile, busfile):
    starttime = time.time()
    tramfile.write("{\n\"results\": [")
    busfile.write("{\n\"results\": [")

    bus_file_size = 0
    tram_file_size = 0

    while bus_file_size < MAX_FILE_SIZE and tram_file_size < MAX_FILE_SIZE:
        try:
            result = helpers.get_active_vehicles(VehicleType.Bus).strip().strip('[').strip(']')
            busfile.write("{},\n".format(result))

            result = helpers.get_active_vehicles(VehicleType.Tram).strip().strip('[').strip(']')
            tramfile.write("{},\n".format(result))

            print("Saved another chunk to file")
            time.sleep(TIMEOUT - ((time.time() - starttime) % TIMEOUT))

            bus_file_size = os.stat(bus_file_name).st_size
            tram_file_size = os.stat(tram_file_name).st_size
        except KeyboardInterrupt:
            tramfile.write("\n ]}")
            busfile.write("\n ]}")
            return


if __name__ == "__main__":
    print("Starting bot.")
    today_string = datetime.datetime.now().strftime('%x')
    with open(tram_file_name, "w") as tram_output, open(bus_file_name, "w") as bus_output:
        saveJsonToFile(tram_output, bus_output)

```

Dodatkowo, należało skorzystać z historycznych danych udostępnionych przez prowadzącego, z 21, 22 oraz 23 marca 2016 roku. Te dane dotyczyły wyłącznie tramwajów, ponieważ dane dotyczące autobusów mogły jeszcze nie być dostępne.

Na podstawie tych danych, należało dojść do interesujących wyników, ze szczególnym uwzględnieniem przedstawienia zebranych danych w postaci map. Duża część wstępnej obróbki danych została wykonana w języku R. Niektóre z wykresów również zostały wykonane w R, ponieważ wykorzystywana biblioteka do rysowania wykresów w Pythonie miała problem z dużą ilością danych.

Główną część analizy stanowią mapy, na których przedstawiony został ruch pojazdów w zależności od różnych czynników. Dodatkowo, w przypadku tramwajów, oszacowany został średni czas dojazdu do centrum Warszawy z wybranych dzielnic, a następnie został on porównany z wartościami podanymi przez ZTM. Dla autobusów, obliczony został całkowity dystans pokonany przez pojazdy w ciągu wybranych okresów czasu, co pozwoliło na oszacowanie kosztów utrzymania komunikacji miejskiej.

# Tramwaje

Do analizy ruchu tramwajów w Warszawie wykorzystano dane zebrane przez naszego bota w niedzielę 14. maja oraz poniedziałek 15. maja. Wybrano te dwa dni, aby możliwe było sprawdzenie różnic w kursowaniu tramwajów w weekend oraz dzień powszedni. Ponadto analizie poddano dane z dwóch dni marca 2016 roku, dostarczone przez prowadzącego.

Dane zbierane były z dwóch endpointów API udostępnianego przez ZTM. Zaletą pierwszego endpointu był kilkukrotnie mniejszy czas między aktualizacją danych o położeniu, z kolei drugi posiadał informację logiczną o tym, czy dany tramwaj jest niskopodłogowy.

Dane po pobraniu zostały poddane obróbce, która usunęła z ramek danych pojazdy o  niepoprawnej geolokacji lub błędnej dacie.

## Sieć tramwajowa

<img src="data\14-05\trams-14-filtered.png"/>
<center style="padding-top:10px">Rysunek x: Sieć tramwajowa Warszawy, na podstawie danych z 14.05.2017 (weekend).</center>    
<img src="data\15-05\trams-15-filtered.png"/>
<center style="padding-top:10px">Rysunek x: Sieć tramwajowa Warszawy, na podstawie danych z 15.05.2017 (dzień powszedni).</center>

Widoczna jest asymetria sieci tramwajowej - jest ona znacznie gęstsza w lewobrzeżnej części miasta. Oczywistym powodem jest trudność w "przeprawie" tramwajów przez Wisłę. Aktualnie realizowana jest ona przez następujące mosty:

<div class="datagrid">
<table>
<thead>
    <tr><th>Nazwa mostu</th><th>Liczba torów tramwajowych</th>
</thead>
<tbody>
    <tr><td>Most Poniatowskiego</td><td>2</td></tr>
    <tr><td>Most Gdański</td><td>2</td></tr>
    <tr><td>Most Śląsko-Dąbrowski</td><td>2N → 2</td></tr>    
    <tr><td>Most Świętokrzyski</td><td>0(2)</td></tr>
    <tr><td>Most Północny</td><td>(2)</td></tr>
</tbody>
</table>
</div>

* W nawiasach przedstawiono ilość planowanych pasów ruchu.
* Strzałkami oznaczono kolejne zmiany ilości pasów.
* Literą N oznaczono pasy niewydzielone, tj. tory tramwajowe niewydzielone z jezdni lub drogi rowerowe niewydzielone z chodnika.

Dane za: [Wikipedia](https://pl.wikipedia.org/wiki/Mosty_w_Warszawie)

Powyższe mapy można skonfrontować z dostępną na Wikipedii mapą sieci tramwajowej stolicy sprzed roku 1939:
<img src="historical_tram_map.png" width=50% />

## Punkty krytyczne sieci tramwajowej

Do wyznaczenia krytycznych punktów sieci tramwajowej posłużyliśmy się heatmapami:

In [24]:
from IPython.display import HTML, display
display(HTML(
    "<table><tr><td><img src='data/14-05/trams-heatmap.png'/>"+
    "</td><td><img src='data/15-05/trams-heatmap.png'/></td></tr></table>"))

,


<center>Rysunek x: Po lewej heatmapa dla 14.05.2017 (niedziela), po prawej dla 15.05.2017 (poniedziałek)</center>

Krytycznymi punktami są z pewnością przystanki Dworzec Centralny, Centrum oraz przystanki w ich okolicy.  
Zauważyć należy również stosunkowo zwiększone natężenie ruchu na Ochocie, w okolicy skrzyżowania ulic Bitwy Warszawskiej 1920r. oraz Banacha, gdzie spotykają się linie obsługiwane przez zajezdnię na Okęciu oraz zajezdnię przy Banacha.   Natężenie stopniowo spada aż do Placu Zawiszy, gdzie część tramwajów jedzie na wschód w kierunku Centrum, a pozostałe na północ.  
Innym ważnym punktem na tramwajowej mapie Warszawy jest Rondo "Radosława", gdzie znajduje się istotny punkt przesiadkowy w stronę Pragi i regularnie kursuje aż 8 linii tramwajowych.
Po lewej stronie Wisły najistotniejsze wydają się przystanki przy Stadionie Narodowym.
Oczywistym faktem jest, że szczególnie wyróżniają się na mapach zajezdnie tramwajowe.

Jeśli chodzi o różnice pomiędzy heatmapą z niedzieli a heatmapą z poniedziałku, to (poza ogólnie większym natężeniem ruchu) zaobserwować można zwiększenie znaczenia zajezdni Mokotów przy skrzyżowaniu Woronicza i Wołoskiej, a także istotne zwiększenie częstości kursowania w kierunku Żoliborza i Bielan.

## Czasy dojazdu

In [3]:
times = pd.read_csv('data/15-05/trams3-filtered/time-data.csv')
times = times.reindex_axis(times.mean().sort_values().index, axis=1)
means = times.mean()
stds = times.std()

timetable_times = [21, 17, 16, 20, 29, 29]

trace1 = go.Bar(
        x = list(pd.DataFrame(means).index),
        y = means,
        name = "Średni czas przejazdu",
        error_y=dict(
            type='data',
            array=stds,
            visible=True
        )
    )

trace2 = go.Bar(
        x = list(pd.DataFrame(means).index),
        name = "Czas przejazdu wg ZTM",
        y = timetable_times,
    )

data = [trace1, trace2]
layout = go.Layout(
    title='Czas dojazdu tramwajem na Dworzec Centralny z wybranych dzielnic w godzinach szczytu',
    barmode='group',
        xaxis=dict(
        title='Dzielnica'
    ),
    yaxis=dict(
        title='Czas [min]'
    )
)
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')


<center style="padding-top:10px">Rysunek x: Wykres zależności czasu dojazdu od dzielnicy, poniedziałek 15.05.2017</center>

W celu utworzenia powyższego wykresu, rozważyliśmy najważniejsze przystanki z każdej dzielnicy i wybraliśmy dla każdej dzielnicy po jednym - takim, aby w przybliżeniu odegłości od przystanku docelowego były równe. Za docelowy przystanek uznaliśmy Dworzec Centralny. Z wyborem tym można polemizować, gdyż w większości powyższych dzielnic znajdują się punkty przesiadkowe, np. stacje kolei miejskiej lub metra, jednak nasz wybór znacząco ułatwia analizę.

Zaobserwować można rozbieżności rzędu 2-3 minut między rozkładowymi czasami przejazdów, a czasami zmierzonymi. Ciekawym przypadkiem jest Ochota, dla której odchylenie standardowe osiąga wartość ponad 6 minut, co stanowi ponad 30% całkowitego czasu przejazdu. Prawdopodobnie wiąże się to z trasą linii kursujących na tym odcinku - muszą one przeciąć dwa ruchliwe węzły w okolicy ul. Bitwy Warszawskiej 1920 r. i Placu Zawiszy. W pesymistycznym przypadku cykle sygnalizacji drogowej mogą tam mieć stosunkowo duży wpływ na ewentualne opóźnienia.

Należy także pamiętać, że wykres ten nie prezentuje optymalnego czasu dojazdu, gdyż często wykorzystanie **tylko** tramwaju jest wolniejsze niż np. podróż metrem.

In [4]:
df = pd.read_csv("data/23-05/tram-counts.csv")

total_count = df['All']
lowfloor = df['Lowfloor']
hours = [7, 8, 9, 10, 11, 12]
    
trace1 = {
    'y': lowfloor,
    'x': hours,
    'name': 'Liczba niskopodłogowych',
    'type': 'bar'
}
trace2 = {
    'y': total_count,
    'x': hours,
    'name':'Całkowita liczba przejazdów',
    'type': 'bar'
}

data = [trace1, trace2]
layout = {
    'title': 'Liczba przejazdów i liczba przejazdów tramwajami niskopodłogowymi we wszystkich przejazdach w ciągu danej godziny',
    'yaxis': {'title':'Liczba niskopodłogowych'},
    'barmode': 'relative'
}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center style="padding-top:10px">Całkowita liczba przejazdów podczas danej godziny oraz liczba przejazdów realizowanych składami niskopodłogowymi w tej godzinie</center>

Zgodnie z przewidywaniami, liczba przejazdów osiąga wartość maksymalną dla zakresu godzin 8-9, czyli godzin szczytu, kiedy młodzież jeździ do szkół, a dorośli do pracy. W ruchu utrzymywana jest w przybliżeniu podobna liczba pojazdów niskopodłogowych, zatem ich odsetek rośnie, kiedy całkowita liczba przejazdów maleje - z tras usuwane są starsze pojazdy.

In [5]:
from IPython.display import display, HTML
df = pd.DataFrame({'Godzina': hours, 'Odsetek niskopodłogowych': round(lowfloor / total_count, 3) * 100})
HTML(df.to_html(index=False))

Godzina,Odsetek niskopodłogowych
7,55.3
8,55.4
9,57.1
10,62.7
11,65.7
12,65.7


In [33]:
df = pd.read_csv("data/23-05/tram-counts-by-line.csv")

lines = df['Line'][:-1]
total_count = df['All'][:-1]
lowfloor = df['Lowfloor']
    
trace1 = {
    'y': 100 * (lowfloor / total_count),
    'x': lines,
    'name': 'Odsetek niskopodłogowych',
    'type': 'bar'
}

data = [trace1]
layout = {
    'title': 'Odsetek pojazdów niskopodłogowych na danej linii',
    'yaxis': {'title':'Odsetek pojazdów niskopodłogowych'},
    'xaxis': {'title':'Numer linii'}
}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

Jak widać, część linii wykorzystuje jedynie pojazdy niskopodłogowe, jednak na najbardziej obleganych trasach, np. 7 lub 9, priorytetem jest duża częstotliwość przejazdów, stąd jedynie około 50% użycie pojazdów niskopodłogowych.

In [6]:
df = pd.read_csv("data/15-05/tram-counts.csv")

trace1 = go.Scatter(
    x=df["Hour"][8:],
    y=df["Count"][8:],
    name='Liczba przejazdów',
    line = dict(dash = 'dot'),
)

data = [trace1]
layout = go.Layout(
    title='Liczba przejazdów w ciągu godziny',
    yaxis=dict(
        title='Liczba przejazdów'
    ),
    xaxis=dict(
        title='Godzina'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic')

<center style="padding-top:10px">Rysunek x: Całkowita liczba przejazdów w ciągu godziny</center>

Widoczne są godziny szczytu, podczas których ZTM udostępnia nawet o 35% więcej tramwajów niż podczas dziennego minimum o godzinie 11 (minimum wyznaczone po odrzuceniu godzin po 21).

# Autobusy

Zebrane dane na temat ruchu autobusów pochodzą z 14 (noc i dzień) oraz 15 maja 2017 roku. Dostępne informacje 
dotyczące autobusów to:

* położenie geograficzne
* czas, w którym zarejestrowano daną pozycję
* numer linii
* numer brygady

Na podstawie numeru linii i brygady można jednoznacznie zidentyfikować dany autobus - w danym momencie porusza się tylko jeden autobus konkretnej linii z danym symbolem brygady. 

Autobusy możemy podzielić na pewne kategorie, które później posłużą do analizy ich ruchu. Te kategorie to:

* linie zwykłe (102 - 262)
* linie zwykłe okresowe (300 - 397)
* linie przyspieszone (500 - 527)
* linie przyspieszone okresowe (401 - 414)
* linie strefowe (700 - 743)
* linie strefowe uzupełniające (L-1 - L40)
* linie strefowe okresowe (800)
* linie ekspresowe (E-1 - E-9)
* linie nocne (N01 - N95)
* linie specjalne (900)

Dane otrzymane z API zostały wstępnie przefiltrowane, aby odrzucić różne błędy (jak np. zła pozycja pojazdu, lub niepoprawna data). Następnie zostały pogrupowane według powyższych kategorii. Dodatkowo, osobno pogrupowane zostały dane dla każdego konkretnego pojazdu (identyfikowanego przez parę numer linii, symbol brygady). Pozwoli to na wyznaczenie dystansu pokonywanego przez autobusy w wybranem okresie czasu, a także na wyznaczenie średniej prędkości każdej linii.
 
Pozycje autobusów z każdej z powyższych grup możemy przedstawić na mapie, która pokaże jaki obszar pokrywają. Na początek rozważmy zdecydowanie najliczniejszą grupę - linie zwykłe.

## Linie zwykłe

<img src="data\14-05\buses-normal14-filtered.png"/>

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy zwykłe 14 maja 2017 roku.</center>

<img src="data\15-05\buses-normal15-filtered.png"/>

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy zwykłe 15 maja 2017 roku.</center>

In [7]:
distance = pd.concat([pd.read_json('file:data/14-05/busesNormalCount.json'), 
                      pd.read_json('file:data/15-05/busesNormalCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,566
15.05.2017,935


<center style="padding-top:10px">Tabela x: Liczba autobusów linii zwykłych w ruchu w wybranych dniach. </center>

Pokonane trasy wyglądają bardzo podobnie. Warto zwrócić uwagę na jeden istotny fakt - 14 maj to niedziela, natomast 15 maj to poniedziałek. Na powyższych mapach można zauważyć, że niektóre linie nie kursują w weekendy, lub mają zmienioną trasę. Na przykład, na mapie z 15 maja możemy zaobserwować żółtą linię w okolicach Zielonki (prawy górny róg mapy), której nie ma na mapie z 14 maja. Podobnie w okolicach Bielawy (prawy dolny róg mapy), pojawia się zielona linia, której nie było w weekend.
Ponadto, warto zauważyć, że wiekszość tych linii jeździ w Warszawie i jej centrum, a tylko nieliczne kursują do miast należących do aglomeracji warszawskiej. To samo widać w zamieszczonej tabeli - zdecydowanie więcej autobusów kursuje w dzień powszedni niż w weekend.

Następnie sprawdźmy, po jakich obszarach jeżdżą autobusy strefowe, które w przeciwieństwie do zwykłych, powinny częściej kursować do okolicznych miejscowości.

## Linie strefowe

<img src="data\14-05\buses-zone14-filtered.png"/>

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy strefowe 14 maja 2017 roku.</center>

<img src="data\15-05\buses-zone15-filtered.png"/>

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy strefowe 15 maja 2017 roku.</center>

In [8]:
distance = pd.concat([pd.read_json('file:data/14-05/busesZoneCount.json'), 
                      pd.read_json('file:data/15-05/busesZoneCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,88
15.05.2017,152


<center style="padding-top:10px">Tabela x: Liczba autobusów linii strefowych w ruchu w wybranych dniach. </center>

Tak jak przewidywaliśmy, autobusy strefowe raczej nie kursują w centrum Warszawy. Ich głównym zadaniem jest przywożenie 
i odwożenie ludzi z okolicznych miejscowości do Warszawy, a żeby dotrzeć do dokładniejszego celu należy się przesiadać. Podobnie jak w przypadku linii zwykłych, więcej autobusów kursuje w dni powszednie.

## Linie przyspieszone

<img src="data\14-05\buses-fast14-filtered.png"/>

<center> Rysunek x: Trasa przebyta przez autobusy przespieszone 14 maja 2017 roku. </center>

<img src="data\15-05\buses-fast15-filtered.png"/>

<center> Rysunek x: Trasa przebyta przez autobusy przespieszone 15 maja 2017 roku. </center>

In [9]:
distance = pd.concat([pd.read_json('file:data/14-05/busesFastCount.json'), 
                      pd.read_json('file:data/15-05/busesFastCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,142
15.05.2017,274


<center>Tabela x: Liczba autobusów linii przyspieszonych w ruchu w wybranych dniach. </center>

Podobnie jak w przypadku linii zwykłych, można zauważyć linie, które nie kursowały w weekend, lub miały zmienioną trasę. Warto zwrócić uwagę na długość tras pokonywanych przez pojedyncze autobusy. Kursują one po liniach zdecydowanie dłuższych niż autobusy zwykłe, dlatego muszą jeździć szybciej, co można osiągnąć na przykład przez mniejsze zagęszczenie przystanków na trasie autobusu.

## Linie przyspieszone okresowe

<img src="data\14-05\buses-fast-periodic14-filtered.png" />

<center style="padding-top:10px"> Rysunek x: Trasa przebyta przez autobusy przespieszone okresowe 14 maja 2017 roku. </center>

<img src="data\15-05\buses-fast-periodic15-filtered.png" />

<center style="padding-top:10px"> Rysunek x: Trasa przebyta przez autobusy przespieszone okresowe 15 maja 2017 roku. </center>

In [10]:
distance = pd.concat([pd.read_json('file:data/14-05/busesFastPeriodicCount.json'), 
                      pd.read_json('file:data/15-05/busesFastPeriodicCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,10
15.05.2017,75


<center style="padding-top:10px">Tabela x: Liczba autobusów linii przyspieszonych okresowych w ruchu w wybranych dniach. </center>

Podobnie jak autobusy przyspieszone, autobusy przyspieszone okresowe również kursują po dłuższych trasach. Tych linii jest jednak zdecydowanie mniej, zwłaszcza w weekend, kiedy to przez cały dzień, kursowało zaledwie 10 autobusów jednej linii.

## Linie nocne

<img src="data\14-05\buses-night14-filtered.png" />

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy nocne 14 maja 2017 roku. </center>

In [11]:
distance = pd.read_json('file:data/14-05/busesNightCount.json')[0].tolist()
distance = pd.DataFrame({'Dzień':'14.05.2017', 'Liczba autobusów':distance}, columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,151


<center style="padding-top:10px">Tabela x: Liczba autobusów linii nocnych w ruchu w wybranych dniach. </center>

Autobusy nocne również pokonują długie trasy, ale kursują rzadziej niż zwykłe. Warto zauważyć, że większość z tych linii przejeżdża przez centrum Warszawy, oraz kończy swoją trasę w okolicznych miejscowościach. Oczywiście w tym przypadku krótkie trasy nie mają sensu. W nocy komunikacją miejską porusza się zdecydowanie mniej osób, więc nie trzeba tworzyć krótkich tras, kursujących często, aby zabrać wszystkich zainteresowanych. Zazwyczaj, autobusy tych linii zabierają mniej ludzi na pojedynczych przystankach, ale pokrywają zdecydowanie większy obszar od autobusów dziennych.

## Pokonany dystans

In [12]:
distance = pd.concat([pd.read_json('file:data/14-05/busDistance-night.json'),
                     pd.read_json('file:data/14-05/busDistance-day.json'),
                     pd.read_json('file:data/15-05/busDistance-day.json')])

distanceList = [x/1000 for x in distance[0].tolist()]

index = ['14.05.2017 - noc', '14.05.2017 - dzień (weekend)', '15.05.2017 - dzień']
trace1 = go.Bar(
        x = index,
        y = distanceList,
        name = "Pokonany dystans"
    )
layout = go.Layout(
    title='Całkowity dystans w kilometrach pokonany przez autobusy w zależności od dnia',
    barmode='group',
    xaxis=dict(
        title='Dzień'
    ),
    yaxis=dict(
        title='Dystans [km]'
    )
)

data = [trace1]

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center> Rysunek x: Wykres pokonanego dystansu przez autobusy w zależności od dnia </center>

In [29]:
distance = pd.DataFrame({'Dzień':index, 'Dystans [km]':[round(dist) for dist in distanceList]}, columns=['Dzień', 'Dystans [km]'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Dystans [km]
14.05.2017 - noc,49676
14.05.2017 - dzień (weekend),134409
15.05.2017 - dzień,337064


<center> Tabela x: Całkowity dystans w kilometrach, pokonany przez autobusy w wybrane dni. </center>

Jak wcześniej zauważyliśmy, zdecydowanie więcej autobusów kursuje w dni powszednie. To samo możemy zauważyć w powyższej tabeli - dystans pokonany przez autobusy w dzień powszedni jest większy, niż suma dystansów pokonanych przez autobusy w trakcie dnia oraz w nocy, w jeden dzień weekendu. Na podstawie tych danych, możemy dokonać pewnego oszacowania. 

Wiemy jaki dystans pokonują autobusy. Możemy też, w przybliżeniu, określić średnie spalanie pojazdów. Najpopularniejszy autobus w Warszawie - Solaris Urbino 18 - według różnych doniesień, spala od 40 do 45 litrów na 100 kilometrów. Jest to uznawane za stosunkowo niskie spalanie, biorąc pod uwagę jego rozmiar. Inne modele mają mniejsze spalanie (niektóre w okolicach 30 litrów na 100 kilometrów), a inne większe (nawet do 60l/100km). Ponieważ ciężko znaleźć dokładne, oficjalne dane oraz nie znamy odległości pokonanych przez każdy z modeli, oszacujmy średnią wartość spalania jako 40 litrów na 100 kilometrów. 

Podobnie nie wiemy, jakie są koszty zakupu paliwa. Ponieważ są to ilości hurtowe (co za chwilę się okaże), możemy założyć, że koszt jest znacznie niższy, niż dla normalnego człowieka na stacji benzynowej. Przyjmijmy 3 złote na litr benzyny. Następnie, dla 15.05.2017, możemy dokonać następujących obliczeń:
                    

<center>337063 \* 40/100 = 134825.2 litrów paliwa **dziennie**, co przekłada się na          
134825.2 \* 3 = **404475.6 złotych**</center>

Analogicznie dla 14.05.2017 w nocy, jest to 59610 złotych, oraz w dzień 161290.8 złotych dziennie. Oczywiście są to tylko szacunkowe koszty - w powyższych obliczeniach nie braliśmy pod uwagę faktu, że niektóre z autobusów są zasilane elektrycznie, lub jeżdżą na gaz. Zdecydowaną większość stanowią jednak pojazdy benzynowe. Oprócz tego, nie braliśmy pod uwagę innych, dodatkowych kosztów, jak np. serwisowanie pojazdów, czy chociażby pensje dla kierowców. Rzeczywisty koszt utrzymania autobusów miejskich może się różnić, jednak powinien to być ten sam rząd wielkości, co pozwala zobrazować, jak kosztowna jest komunikacja miejska. 

Obliczony koszt możemy przedstawić jeszcze w inny sposób - najtańszy bilet normalny, 20-minutowy, kosztuje w Warszawie 3.40 złotych. Aby zwrócił się oszacowany koszt utrzymania w dzień powszedni, należy sprzedać conajmniej 118964 takich biletów, w ciągu jednego dnia.

## Średnia prędkość

Na podstawie otrzymanych danych, możemy również określić, w przybliżeniu, średnią prędkość dla każdej z linii. W tym celu, autobusy zostały pogrupowane według numeru linii i brygady. Następnie, dla każdego zidentyfikowanego w ten sposób pojazdu, została policzona prędkość na każdym z odcinków (różnica odległości pomiędzy kolejnymi zarejestrowanymi pozycjami), a z tych prędkości wyciągnięta została średnia. Jest zbyt wiele pojedynczych linii, aby przedstawić prędkość każdej z nich, ale posłużymy się wcześniej zdefiniowanymi grupami linii autobusowych i wyznaczymy średnią prędkość dla każdej grupy.

In [14]:
day14speed = pd.concat([pd.read_json('file:data/14-05/busSpeed-day-grouped.json'),
                     pd.read_json('file:data/14-05/busSpeed-night-grouped.json')])

day15speed = pd.read_json('file:data/15-05/busSpeed-day-grouped.json')

trace1 = go.Bar(
        x = day14speed['Type'],
        y = day14speed['V'],
        name = "14.05.2017"
    )

trace2 = go.Bar(
        x = day15speed['Type'],
        y = day15speed['V'],
        name = "15.05.2017"
    )

layout = go.Layout(
    title='Średnia prędkość wybranych linii autobusowych',
    barmode='group',
    xaxis=dict(
        title='Rodzaj linii'
    ),
    yaxis=dict(
        title='Prędkość [km\h]'
    )
)

data = [trace1, trace2]

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center> Rysunek x: Średnia prędkość wybranych linii autobusowych w zależności od dnia </center>

In [15]:
day15speedModified = pd.concat([day15speed, pd.DataFrame({'Type':'Nocne', 'V':'-'}, index = [0])])

speed =  pd.DataFrame({ 'Rodzaj linii':day14speed['Type'], '14.05.2017':day14speed['V'],
                      '15.05.2017':day15speedModified['V']}, columns=['Rodzaj linii', '14.05.2017', '15.05.2017'])
HTML(speed.to_html(classes=['table'], index=False))

Rodzaj linii,14.05.2017,15.05.2017
Zwykle,26.4748,25.1964
Strefowe,31.4119,28.7053
Przyspieszone,27.0149,24.5094
Przyspieszone okresowe,27.1122,25.1269
Nocne,30.2197,-


<center>Tabela x: Średnia prędkość wybranych linii autobusowych w zależności od dnia </center>

Łatwo można zauważyć, że średnie prędkości w weekend są wyższe niż w dzień. Podobnie w nocy możemy się spodziewać większej średniej prędkości. Nie powinno to zaskakiwać, ponieważ w dni powszednie ruch na ulicach jest dużo większy, co powoduje, że autobusy stoją w korkach, przez co często się spóźniają. Tak samo wygląda sytuacja, jeśli chodzi o autobusy strefowe, które poruszają się głównie z dala od centrum Warszawy, oraz po okolicznych miejscowościach, gdzie ruch jest mniejszy. Co ciekawe, autobusy przyspieszone nie muszą jeździć szyciej niż autobusy zwykłe. Oczywiście nasze obliczenia są w pewnym stopniu nie dokładne i mogły wystąpić pewne błędy, ale różnica między prędkościami tych linii nie jest duża. Autobusy przyspieszone nie zatrzymują się na niektórych przystankach, ale przy wzmożonym ruchu również będą stać w korkach, co może spowodować wyrównanie tych prędkości.